# Pipeline building Meta-Model of EPIC Response Curves

 Step 0: Choose data subset, investigate feature importance for the given subset
 
 Step 1: Identify importance of variables 
 
 Look at a variety of models and methods to determine the best feature importance 
 
 Step 2: Build Response Curve 
  
 Use matching to build response curves 
 
  Exact match based on EPIC experiments where they exist (same simulation ID x year, different treatments), propensity score match when set experiments do not exist match (different simulation ID x years, same x variables, different treatments)
  
 Choose the form of the response curve - spheroid 
 
 Step 3: Build Meta-Model
 
 build a machine learning model which learns the relationship between inputs and response curve shape - ensure there are parameters which can be tuned
 
 Step 4: Test if tuned meta-model can predict yields  
 
 test on czech data 


In [1]:
## import statements 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from functools import reduce
import sklearn
import matplotlib
import os
import warnings
from multiprocessing import Pool
from IIASA_22_fxns import get_N_exp, get_season_info, split_data, get_gs_climate, tt_split_scale, random_forest, yield_run_data
import tqdm
warnings.filterwarnings('ignore')

In [2]:
## datasets that may be used 
# simulation location data - X, Y, SimUID
loc = pd.read_csv("//Users//taraippolito//Desktop//Desktop_Tara’s_MacBook_Pro//EPIC_local//SimUID_Pts_210_240.txt", sep = ",")

# simUID site data
site_df = pd.read_csv("//Users//taraippolito//Desktop//Desktop_Tara’s_MacBook_Pro//EPIC_local//_SimUData//SimUID_List.txt", sep = ";")

# simulation units, all climate, site data - YEAR IS SEPARATE
obs_df = pd.read_csv("//Users//taraippolito//Desktop//Desktop_Tara’s_MacBook_Pro//EPIC_local//_SimUData//SimUID_static+clim.csv")

# # clusters
# clust = pd.read_csv("//Users//taraippolito//Desktop//Desktop_Tara’s_MacBook_Pro//EPIC_local//_SimUData//climate_PCA_x_simUID.csv")

# # Growing season climate variables 
# GS_clim = pd.read_csv("//Users//taraippolito//Desktop//Desktop_Tara’s_MacBook_Pro//EPIC_local//CORN//C_BAU_R00_GSclim.csv") 

# climate files 
# Import simulation unit data and all weather data 
pet_df = pd.read_csv("//Users//taraippolito//Desktop//Desktop_Tara’s_MacBook_Pro//EPIC_local//_Weather//CORN_dyn_rf_BAU_R00_PET.txt", sep = ",")
prcp_df = pd.read_csv("//Users//taraippolito//Desktop//Desktop_Tara’s_MacBook_Pro//EPIC_local//_Weather//CORN_dyn_rf_BAU_R00_PRCP.txt", sep = ",")
rad_df = pd.read_csv("//Users//taraippolito//Desktop//Desktop_Tara’s_MacBook_Pro//EPIC_local//_Weather//CORN_dyn_rf_BAU_R00_RAD.txt", sep = ",")
tmin_df = pd.read_csv("//Users//taraippolito//Desktop//Desktop_Tara’s_MacBook_Pro//EPIC_local//_Weather//CORN_dyn_rf_BAU_R00_TMN.txt", sep = ",")
tmax_df = pd.read_csv("//Users//taraippolito//Desktop//Desktop_Tara’s_MacBook_Pro//EPIC_local//_Weather//CORN_dyn_rf_BAU_R00_TMX.txt", sep = ",")
vpd_df = pd.read_csv("//Users//taraippolito//Desktop//Desktop_Tara’s_MacBook_Pro//EPIC_local//_Weather//CORN_dyn_rf_BAU_R00_VPD.txt", sep = ",")
cmd_df = pd.read_csv("//Users//taraippolito//Desktop//Desktop_Tara’s_MacBook_Pro//EPIC_local//_Weather//CORN_dyn_rf_BAU_R00_CMD.txt", sep = ",")

## Step 0: Choose data subset

In [3]:
#### identify data subset to start with 
C = ["CORN"] 
N = ["BAU", "N01", "N50", "N100", "N250"]
# N = ["BAU"]
R = ["R00"]

In [4]:
#### Get season-specific info: start of season, end of season, length of season 
# pull arguments to feed into function in parallel
season_info_args = []
for c in C:
    for n in N: 
        for r in R: 
            season_info_args.append((c,n,r))

In [5]:
#### get epic yield data for each treatment 
yields = []

for arg in season_info_args: 
    yields.append(yield_run_data(arg))

In [35]:
#### run in parallel 
# arguments = season_info_args
# # PARALLEL 
# if __name__ == '__main__':
#     print ("in main.")
#     with Pool(4) as pool:
#         season_info_dfs = list(tqdm.tqdm(pool.imap(get_season_info, arguments), total=len(arguments)))
#         pool.close() 

#### or run simply if data subset is not too big - e.g. just one crop x treatment 
season_info_dfs = []
for arg in season_info_args: 
    season_info_dfs.append(get_season_info(arg))
    print ("season info, done.")

season info, done.
season info, done.
season info, done.
season info, done.
season info, done.


In [36]:
#### Split seasonal info dataframes into smaller dataframes where year, season length, and season start are unique
# run in parallel 
# arguments = season_info_dfs
# # PARALLEL
# if __name__ == '__main__':
#     print ("in main.")
#     with Pool(4) as pool:
#         split_dfs = list(tqdm.tqdm(pool.imap(split_data, arguments), total=len(arguments)))
#         pool.close() 

#### or run simply if data subset is not too big
#### output is a list of lists, first dimension is treatment season info df, second dimension is season start/len split
split_dfs = []
for df in season_info_dfs:
    # add lists to split_df list that come out of split_data
    split_dfs.append(split_data(df))

In [37]:
#### For the split up seasonal info dataframes, calculate growing season variables for a given climate variable 
#### NOT IN PARALLEL AS PARALLEL TAKES LONGER 
climate_dfs = [pet_df, prcp_df, rad_df, tmin_df, tmax_df, vpd_df, cmd_df]
GS_dfs_all_szn = []

# save growing season climate variables for each treatment 
ALLGSclimxtreat = []
# loop over the treatments
for i in range(len(split_dfs)):
    # save growing season climate for each climate variable
    GSclim_dfs = []
    # for each climate variable
    for clim_df in climate_dfs: 
        clim_out_list = []
        # go over all the season len splits in a given treatment and get the climate cols for the seasonal split 
        for df in split_dfs[i]:
            # get season x treatment specific engineered variables
            clim_out_list.append(get_gs_climate((df, clim_df)))
        # for all the splits, concat them vertically then add to the 
        GSclim_dfs.append(pd.concat(clim_out_list))
        print ('clim done')
    # drop duplicate variables from new dataframes 
    concat_clim = pd.concat(GSclim_dfs, axis = 1)
    ready = concat_clim.loc[:,~concat_clim.columns.duplicated()]
    
    ALLGSclimxtreat.append(ready)
    print ("treatment done.")

clim done
clim done
clim done
clim done
clim done
clim done
clim done
treatment done.
clim done
clim done
clim done
clim done
clim done
clim done
clim done
treatment done.
clim done
clim done
clim done
clim done
clim done
clim done
clim done
treatment done.
clim done
clim done
clim done
clim done
clim done
clim done
clim done
treatment done.
clim done
clim done
clim done
clim done
clim done
clim done
clim done
treatment done.


## Step 1: Identify Importance of Variables 

In [6]:
## for testing purposes 
ALLGSclimxtreat = [GS_clim]

In [17]:
#### USE A RANDOM FOREST REGRESSION MODEL TO IDENTIFY FEATURE IMPORTANCE 
#### Build random forest with all features that data you would like to predict has available - 
#### we only want feature importance for features we can use in predictions 

# get train test sets for each treatment 
train_test = []
all_data_sets = []

# for each treatment, get a predictor set and a target set 
for i in range(len(ALLGSclimxtreat)): 
    # pull site specific data 
    # One-hot encode the soil hydrological group 
    HSG_dummy = pd.get_dummies(obs_df.HSG2, prefix = "HSG")
    dummy_add = pd.concat([obs_df, HSG_dummy], axis = 1)
    
    # MERGE THE SEPARATE DATA PIECES TOGETHER 
    predictor_data = pd.merge(dummy_add, ALLGSclimxtreat[i], how = "left", on = ["SimUID", "YR"]).drop(['Unnamed: 0_x', 'Unnamed: 0_y', 'SimU', 'NUTS2',
       'LCF3', 'HRU', 'ELEV_CAT', 'SLP_CAT', 'TEXTURE', 'DTR', 'STONES',
       'ELEV', 'HSG2'], axis = 1)
    predictor_vars = list(predictor_data.columns)[2:]
    target_data = yields[i][['SimUID', 'YR', 'SCEN', 'YLDG', 'BIOM']]
    all_data = pd.merge(predictor_data, target_data, how = "left", on = ["SimUID", 'YR'])
    # save the big datasets
    all_data_sets.append(all_data)
    
    # get train test splits
    X_train, X_test, y_train, y_test = tt_split_scale(all_data, "YLDG")
    # append to list 
    train_test.append([X_train, X_test, y_train, y_test])

finished TT split.
finished fitting scaler.
finished X transformation.
finished y transformation.


In [23]:
# get feature importance for each treatment and scores
feat_imp_list = [] 
scores = []

for tts in train_test: 
    y_predicted, score, feat_imp = random_forest(tts[0], tts[1], tts[2], tts[3], 50, 20)
    feat_df = pd.DataFrame(feat_imp, index = predictor_vars, columns = ['feat_imp']).sort_values('feat_imp', ascending = False)
    
    feat_imp_list.append(feat_df)
    scores.append(score)

In [40]:
feat_imp_list[0].iloc[20:40]

,feat_imp
RADavGS,0.008603
JUN_PRCP,0.007917
AGG_PET,0.007594
AGG_PRCP,0.007409
KS_SUB2,0.007244
FEB_RAD,0.007092
TMNskGS,0.007068
MEAN_RAD,0.006850
JUN_TMN,0.006708
MEAN_PRCP,0.006481


## Step 2: Build Response Curves of Identified Important variables
1. To build response curves with baked in EPIC experiments, use simulation unit and year to look at responses (e.g. N fertilized at SimUID 3 in 1985, with N rates 0, 50, 100, 250, BAU)
2. To build response curves for experiments outside EPIC defined experiments, use matching 

In [ ]:
# Build matched data subsets for each response curve being built 


## Step 3: Build Meta-Model 
1. Meta-model is an ensemble of polynomial regression functions (response curves) 
2. Ensemble base models are weighted according to feature importance derived from random forest 
3. Final output is weighted average from predictions from base models 